In [1]:
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import datetime
import calendar
import time
import matplotlib.pyplot as plt
import math

#from sklearn import linear_model
#from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import linear_model
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

# scaling the input feature
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

from joblib import load
from joblib import dump

In [2]:
df_train = pd.read_csv('crime_data/fullTrainTestCrimeData_train_weather_by_hourgroup_police_rand_2019_06_27.csv')
df_train = df_train.drop(columns=["Unnamed: 0"])

#df_train.boxplot(column="Value")
# remove outliers
ind_to_drop_outlier = df_train[df_train["Value"]>60].index
df_train = df_train.drop(ind_to_drop_outlier, axis=0)

df_valid = pd.read_csv('crime_data/fullTrainTestCrimeData_valid_weather_by_hourgroup_police_rand_2019_06_27.csv')
df_valid = df_valid.drop(columns=["Unnamed: 0"])

df_test = pd.read_csv('crime_data/fullTrainTestCrimeData_tests_weather_by_hourgroup_police_rand_2019_06_27.csv')
df_test = df_test.drop(columns=["Unnamed: 0"])

#df_uniq_id["DATE"] = df_uniq_id["OCCURRED_ON_DATE"].apply(lambda x: x.split(" ")[0])
df_train["Value_Occ"] = df_train["Value"].apply(lambda x: 1 if x>0 else x)
df_valid["Value_Occ"] = df_valid["Value"].apply(lambda x: 1 if x>0 else x)
df_test["Value_Occ"]  = df_test["Value"].apply(lambda x: 1 if x>0 else x)

In [3]:
x_train = df_train[["Hour_Grp","DAY_OF_MONTH","DAY_OF_WEEK_NUM","MONTH","Lat","Long"
                    ,"Temperature(F)","Humidity(%)","Wind Speed(mph)","Precip.(in)",
                    "closest_police_d","closest_mbta_d"]]

x_valid = df_valid[["Hour_Grp","DAY_OF_MONTH","DAY_OF_WEEK_NUM","MONTH","Lat","Long"
                    ,"Temperature(F)","Humidity(%)","Wind Speed(mph)","Precip.(in)",
                    "closest_police_d","closest_mbta_d"]]

x_test = df_test[["Hour_Grp","DAY_OF_MONTH","DAY_OF_WEEK_NUM","MONTH","Lat","Long"
                    ,"Temperature(F)","Humidity(%)","Wind Speed(mph)","Precip.(in)",
                  "closest_police_d","closest_mbta_d"]]

y_train = df_train[["Value_Occ"]]
y_valid = df_valid[["Value_Occ"]]
y_test = df_test[["Value_Occ"]]

## Look at data, how was it built? 
- See code: segment_map_time_reduce_police_random.ipynb
- train is evenly grids 
- valid and test are random around the train grids

## MEAN
### Calculate Mean Values
- for a location, calculate the average over time
- this is calculated on the training sample only
    - think of it as a model which is trained on training sample

In [4]:
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import datetime
import calendar
import time
import matplotlib.pyplot as plt
import math

# parameters setup for segmenting the area
grid_w = 0.5 # mi
grid_deg = round((grid_w/111)*1.60934, 4)

lat_dn = 42.2324
lat_up = 42.3951
lng_dn = -71.1787
lng_up = -70.9636

# start_day = datetime.datetime(2015,6,15)
# last_day = datetime.datetime(2015,9,30)

import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

fp = "/Users/apicharthortiangtham/InsightDS/1_work/2019_06_07_boston_crime_project/bos_land/bos_land.shp"
data_shapemap = gpd.read_file(fp)

# Convert data to global lat long
# EPSG 4326 = WGS84 = https://epsg.io/4326
data_shapemap_geo = data_shapemap.to_crs(epsg=4326) 
multi_polygon = list(data_shapemap_geo["geometry"][0])

def in_MutiPolygon(lat, lng, multi_polygon):
    is_inside = False
    for polg in multi_polygon:
        point = Point(lng, lat)
        is_inside = polg.contains(point)
        if is_inside:
            return is_inside
    return is_inside

a = lat_dn
lat_list=[]
lat_list+=[round(a,4)]
while a <= lat_up:
    a+= grid_deg
    lat_list += [round(a,4)] 
    
b = lng_dn
lng_list=[]
lng_list+=[round(b,4)]
while b <= lng_up:
    b+= grid_deg
    lng_list += [round(b,4)]
print (len(lng_list),len(lat_list))

hour_groups = [1,2,3]

31 24


In [5]:
#def create_mean_feature(df_train):
cnt = 0
data = []
for lng in lng_list:
    cnt +=1 
    print ("count : ", cnt )
    for lat in lat_list:
        in_Bos = in_MutiPolygon(lat, lng, multi_polygon)
        if in_Bos:
            for this_hr in hour_groups:
                con1 = (df_train["Lat"] < lat + 0.000001)
                con2 = (df_train["Lat"] > lat - 0.000001)
                con3 = (df_train["Long"] < lng + 0.000001)
                con4 = (df_train["Long"] > lng - 0.000001)
                
                _mean = df_train[ con1&con2&con3&con4 & (df_train["Hour_Grp"] == this_hr)]["Value_Occ"].mean()
                #print (cnt, lng, lat,  _mean)
                dat = { 'Lat': lat,'Long': lng, 
                        'Lat_int': int(lat*1000),'Long_int': int(lng*1000), 
                           'Hour_Grp':this_hr,
                           'mean': round(_mean),
                           'mean_raw': _mean,
                          }
                data += [dat]
train_mean = pd.DataFrame(data)
#    return train_mean
#train_mean = create_mean_feature(df_train)

count :  1
count :  2
count :  3
count :  4
count :  5
count :  6
count :  7
count :  8
count :  9
count :  10
count :  11
count :  12
count :  13
count :  14
count :  15
count :  16
count :  17
count :  18
count :  19
count :  20
count :  21
count :  22
count :  23
count :  24
count :  25
count :  26
count :  27
count :  28
count :  29
count :  30
count :  31


## MEAN prediction for the Test sample : real calculation 
   - since now the validate and test sample have random grid poits
   - we need more caluclation to get mean of these sample 

In [6]:
def get_mean_predict(inp_lat, inp_lng, inp_hr_gr, grid_deg, df_mean_in): 
    #tmp_df["Lat"].iloc[0]
    #tmp_df["Lat"]
    cond2 = df_mean_in["Lat"] < (inp_lat + 1.1*grid_deg)
    cond3 = df_mean_in["Lat"] > (inp_lat - 1.1*grid_deg)
    cond4 = df_mean_in["Long"] < (inp_lng + 1.1*grid_deg)
    cond5 = df_mean_in["Long"] > (inp_lng - 1.1*grid_deg)
    cond6 = df_mean_in["Hour_Grp"] == inp_hr_gr
    tmp_df = df_mean_in[cond2 & cond3 & cond4 & cond5 & cond6]
    #print (inp_lat, inp_lng)
    #print (tmp_df)
    
    min_dist = 200
    dist = 0
    y_dist = 0
    x_dist = 0
    mean_val = 0
    
    for i in range(tmp_df.shape[0]):
        la = tmp_df["Lat"].iloc[i]
        ln = tmp_df["Long"].iloc[i]
        
        y_dist = abs(inp_lat-la) *(111/1.60934)
        x_dist = abs(inp_lng-ln) *(111/1.60934)
        dist = math.sqrt((y_dist**2) + (x_dist**2))
        
        if dist < min_dist:
            min_dist = dist
            mean_val = tmp_df["mean"].iloc[i]
            #print ("min_dist ", min_dist, la, ln, mean_val)
    
    return mean_val

In [7]:
print ("Warning: This takes 8 minutes to finish")
start_time = time.time()
#df_test["mean"] = 0
test_sample_meanout = []
for i in range(df_test.shape[0]):
#for i in range(df_test[:500].shape[0]):
    if i%10000 == 0: print (i)
    _mean = get_mean_predict(df_test["Lat"].iloc[i], df_test["Long"].iloc[i], df_test["Hour_Grp"].iloc[i], grid_deg, train_mean)
    test_sample_meanout += [_mean]
    #print ("_mean ", _mean)
print ("Elapsed time: ", str(time.time()-start_time))

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
Elapsed time:  421.33611488342285


In [10]:
y_test_mean = pd.DataFrame({'test_set_cal_mean': test_sample_meanout})
#print (y_test_mean.head())
print ( "MEAN f1_score      : ", f1_score(y_test, y_test_mean))
print ( "MEAN accuracy_score: ", accuracy_score(y_test, y_test_mean))

NameError: name 'test_sample_meanout' is not defined

In [ ]:
# save the results of calculation for later use
#y_test_mean.to_csv('crime_data/test_cal_mean_police_rand_2019_06_27.csv')
y_test_mean.to_csv('crime_data/test_cal_mean_police_rand_xxxxx.csv')

#### Read from saved results

In [4]:
y_test_mean = pd.read_csv("crime_data/test_cal_mean_police_rand_2019_06_27.csv")
y_test_mean = y_test_mean.drop(columns=["Unnamed: 0"])

In [5]:
y_test_true = df_test[["Value_Occ"]]
print ( "MEAN f1_score      : ", f1_score(y_test, y_test_mean))
print ( "MEAN accuracy_score: ", accuracy_score(y_test, y_test_mean))

MEAN f1_score      :  0.5527150217994452
MEAN accuracy_score:  0.8082738702004757


In [6]:
print("Classification Report MEAN")
print(classification_report(y_test, y_test_mean))

Classification Report MEAN
              precision    recall  f1-score   support

           0       0.85      0.91      0.88    125556
           1       0.62      0.50      0.55     39252

    accuracy                           0.81    164808
   macro avg       0.74      0.70      0.72    164808
weighted avg       0.80      0.81      0.80    164808



## Grad Boosted DT
- #### Load model 
- #### or Run training

In [6]:
params = {
    'n_estimators': 100,
    'max_depth': 6,
    'learning_rate': 0.2,
    'criterion': 'friedman_mse',
    'random_state': 42
}

ct = ColumnTransformer([
        ('somename', StandardScaler(), 
         ["Lat","Long","Temperature(F)","Humidity(%)","Wind Speed(mph)","Precip.(in)","closest_police_d","closest_mbta_d"])
    ], remainder='passthrough')

grad_boost_classi_pipe = Pipeline(steps=[('preprocessor', ct),
                      ('classifier', ensemble.GradientBoostingClassifier(**params))])

grad_boost_classi_pipe.fit(x_train, y_train)

/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('somename',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['Lat', 'Long',
                                                   'Temperature(F)',
                                                   'Humidity(%)',
                                                   'Wind Speed(mph)',
                                                   'Precip.(in)',
                                                   'closest_police_d',
                                              

In [7]:
predictions_test = grad_boost_classi_pipe.predict(x_test)
print ("test f1 ", f1_score(y_test, predictions_test))
print ("test accuracy_score ", accuracy_score(y_test, predictions_test))

test f1  0.5277190700699074
test accuracy_score  0.8061077132178049


In [8]:
predictions_valid = grad_boost_classi_pipe.predict(x_valid)
print ("validate f1 ", f1_score(y_valid, predictions_valid))
print ("validate accuracy_score ", accuracy_score(y_valid, predictions_valid))

validate f1  0.5798109482657183
validate accuracy_score  0.8044502044392523


In [9]:
predictions_train = grad_boost_classi_pipe.predict(x_train)
print ("train f1 ", f1_score(y_train, predictions_train))
print ("train accuracy_score ", accuracy_score(y_train, predictions_train))

train f1  0.5787184674965111
train accuracy_score  0.8090214789859601


### Save Model

In [1]:
# save model
# https://www.mikulskibartosz.name/how-to-save-a-machine-learning-model-into-a-file/
toBePersisted = dict({
    'model': grad_boost_classi_pipe,
    'metadata': {
        'name': 'Does a crime occur?',
        'author': 'Apichart H',
        
    }
})

dump(toBePersisted, 'grad_bdt_classi_police_mbta_xxxxxxx.joblib')

NameError: name 'grad_boost_classi_pipe' is not defined